# Traffic state visualization with foulium and heatmap with time


In [ ]:
import pandas as pd

tfs= pd.read_csv('2022_03_Marc_TRAMS_TRAMS.csv', sep=',')
tfs= tfs.loc[(20220325070000<=tfs.data) & (tfs.data<20220325240000)] # select an especific day 
tfs['hour']=0 # add a new column to represent the hour
tfs['min']=0 # add a new column to represent the minute

for index, row in tfs.iterrows():
    strdate= str(row.data)
    row['hour'] =int(strdate[8]+strdate[9]) # get the hour value for every row
    row['min'] = int(strdate[10]+strdate[11]) # get the minute value for every row

# Check if there is traffic information missing for a given time
minutes=[0,5,10,15,20,25,30,35,40,45,50,55]
for hour in range(7,23):
    for min in minutes:
        d= tfs.loc[(hour==tfs['hour']) & (min==tfs['min'])]
        if d.shape[0]==0:
            print(hour,min)

In [ ]:
tfs.head()

In [ ]:
tfs.info()

In [ ]:
sections = pd.read_csv('transit_relacio_trams.csv', sep=',')
sections.head()

In [ ]:
streets = sections["Coordenades"].str.split(',')
streets

In [ ]:
from shapely.geometry import LineString

sections = pd.read_csv('transit_relacio_trams.csv', sep=',')

def reshape(coordinates):
    coordinates = coordinates.split(",")
    reshaped=[]
    for i in range(0, len(coordinates), 2):
        a = (float(coordinates[i]), float(coordinates[i+1]))
        reshaped.append(a)
    return LineString(reshaped)

geometry = sections['Coordenades'].apply(lambda x: reshape(x))
geometry

In [ ]:
import folium
from geopandas import GeoDataFrame

gdf = GeoDataFrame(sections, crs="EPSG:4326", geometry=geometry)
barcelona_map = folium.Map([41.40,2.17], zoom_start=12) # create a folium map
folium.GeoJson(gdf).add_to(barcelona_map) # plot the street sections onto the map
# plot map
barcelona_map

In [ ]:
auxtfs= tfs.loc[(tfs['hour']==15) & (tfs['min']==0)].reset_index() # Get traffic state of a specific hour
data = pd.concat([auxtfs, sections], axis=1) # merge the traffic state information with section geometry
data['geometry'] = geometry
merged_gdf = GeoDataFrame(data, crs="EPSG:4326", geometry=data['geometry'])
merged_gdf = merged_gdf[["estatActual", "Tram", "geometry"]]
merged_gdf

In [ ]:
import branca
import branca.colormap as cm

colorscale = cm.LinearColormap(colors=['silver','cyan','lawngreen','yellow','orange','red', 'black'], index=[0,1,2,3,4,5,6],vmin=0,vmax=6)
colorscale.caption = "Traffic state of the sections"

def style_function(feature):
    return { 'color': colorscale(int(feature['properties']['estatActual'])) }

barcelona_map = folium.Map([41.40,2.17], zoom_start=12) # create a folium map
folium.GeoJson(merged_gdf, style_function=style_function).add_to(barcelona_map)
barcelona_map.add_child(colorscale)

# plot map
barcelona_map

In [ ]:
from folium.plugins import HeatMapWithTime
from branca.element import Figure

coords_1=[] # coordinates of the vhehicles at different time period
# assign time period to traffic states recorded at the same time
tfs['period']=0
tfs['period']= sorted([i for i in range(1, int(tfs.shape[0]/len(tfs.idTram.unique()))+1)] * len(tfs.idTram.unique()))

for m in tfs['period'].unique():
    s=[] # traffic state of every period
    temp= tfs.loc[tfs['period']==m]
    index = 0
    for n in tfs.idTram.unique():
        # we add 1 to the traffic state of every section, therefore, sections with traffic state 0 (no data)
        # will be considered as fluent traffic state
        traffic= (int(temp['estatActual'].loc[temp['idTram']==n].values)+1) 
        pos = 0
        while pos < len(sections.Coordenades.iloc[index].split(','))-2:
            # start and end point of a line segment
            start= [float(sections.Coordenades.iloc[index].split(',')[pos+1]), float(sections.Coordenades.iloc[index].split(',')[pos])]
            end = [float(sections.Coordenades.iloc[index].split(',')[pos+1+2]), float(sections.Coordenades.iloc[index].split(',')[pos+2])]
            
            # assign "vehicles" to the line segment according to the traffic situation
            x= (end[0]- start[0])/traffic 
            y= (end[1]- start[1])/traffic
            for i in range(traffic):
                start[0]+=x
                start[1]+=y
                s.append([float(start[0]), float(start[1])])
            pos+=2
        index += 1
    coords_1.append(s)

# add the vehicle coordinates as heatmap dots to the map
heatmap=Figure(width=850,height=550)
barcelona_map=folium.Map(location=[41.40,2.17],zoom_start=10)
heatmap.add_child(barcelona_map)
HeatMapWithTime(coords_1,radius=5,auto_play=True,position='bottomright').add_to(barcelona_map)
barcelona_map